# Project: DaXiGua Agency

In [1]:
import copy
import numpy as np

In [2]:
class Position(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Velocity(object):
    def __init__(self, vx, vy):
        self.x = vx
        self.y = vy

In [3]:
class Ball(object):
    
    def __init__(self, position, velocity, radius, color=(255, 255, 255)):
        """
        position: numpy array
        velocity: numpy array
        """
        self.position = position
        self.velocity = velocity
        self.radius = radius
        self.color = color
    
    def __str__(self):
        return f"Position, x: {self.position[0]}, y: {self.position[1]} \nVelocity, vx: {self.velocity[0]}, vy: {self.velocity[1]} \nRadius: {self.radius}"

In [237]:
import cv2 as cv


class State(object):
    
    def __init__(self, screen_x, screen_y, balls, endline, bg_color=(0, 165, 255)):
        """
        screen_x: float, the width of the screen
        screen_y: float, the height of the screen
        balls: List[Ball], the  list of balls in the screen
        """
        self.screen_x = screen_x
        self.screen_y = screen_y
        assert endline < screen_y
        self.endline = endline
        self.balls = balls
        self.bg_color = bg_color
        self.canvas = InitCanvas(self.screen_x, self.screen_y, color=bg_color)

    def reset_canvas(self):
        self.canvas = InitCanvas(self.screen_x, self.screen_y, color=self.bg_color)

    def plot_state(self):
        """Plot the State with an image"""
        cv.line(self.canvas,
                (0, self.screen_y - self.endline),
                (self.screen_x, self.screen_y - self.endline),
                (0, 0, 255), thickness=2)
        for ball in self.balls:
            print(ball)
            cv.circle(self.canvas,
                      (int(ball.position[0]), int(self.screen_y-ball.position[1]) ),
                      ball.radius, ball.color, -1)

        cv.imshow('state', self.canvas)
        cv.waitKey(0)
        cv.destroyWindow('state')


def InitCanvas(width, height, color=(255, 255, 255)):
    canvas = np.ones((height, width, 3), dtype="uint8")
    canvas[:] = color
    return canvas

In [306]:
def evaluate_by_gravity(state):
    """
    state: State, the initial state
    return: State, the final converged state
    
    implement the movement of the balls in the state by the effect of gravity
    """
    
    g = -9.8
    amortize_factor = 0.99  # further tuning needed
    collision_factor = 0.09  # further tuning needed

    screen_limit = np.array([state.screen_x, state.screen_y])

    dt = 0.01  # time step of evaluation
    t = 0

    frames = [state]  # store the frames of evaluation
    converged = False  

    balls = state.balls
    
    count = 0
    
    while not converged:

        N = len(balls)

        for i in range(N):
            b = balls[i]
            f = np.array([0, 1*g])

    #         Update the velocity
            b.velocity = (1 - amortize_factor * dt) * b.velocity + dt * f

    #         Update the position
            b.position = b.position + b.velocity * dt

    #         Check collision with borders
            for j in range(2):
                if b.position[j] < b.radius:
                    b.velocity[j] = 0
                    b.position[j] = b.radius          

                if b.position[j] > screen_limit[j] - b.radius:
                    b.velocity[j] = 0
                    b.position[j] = screen_limit[j] - b.radius

    #         Check collisions between balls
        for i in range(N):
            for j in range(i+1, N):
                ball_1 = balls[i]
                ball_2 = balls[j]
                
                
                if ( np.linalg.norm(ball_1.position - ball_2.position) <= ball_1.radius + ball_2.radius ):
                    
                    m1 = ball_1.radius
                    m2 = ball_2.radius
                    
                    mid_point = (1/(m1 + m2)) * (m2 * ball_1.position +  m1 * ball_2.position)
                    u = (ball_2.position - ball_1.position)  # uniform vector from ball 1 to ball 2
                    u = u / np.linalg.norm(u)
                    #  Update the positions after collision
                    ball_1.position = mid_point - ball_1.radius * u
                    ball_2.position = mid_point + ball_2.radius * u
                    
                    # Update the velocity of balls after collsion
                    # divide the velocity to two dimension : u and n
                    
                    v1_u = np.dot(ball_1.velocity, u) * u
                    v1_n = ball_1.velocity - v1_u
                    
                    v2_u = np.dot(ball_2.velocity, u) * u
                    v2_n = ball_2.velocity - v2_u
                    
                    # the velocity of direction n does not change, but in direction u they exchange

                    
                    v1_u_after = collision_factor * ((m1 - m2) * v1_u + 2 * m2 * v2_u)/(m1 + m2)
                    v2_u_after = collision_factor * ((m2 - m1) * v2_u + 2 * m1 * v1_u)/(m1 + m2)
                    
                    ball_1.velocity = amortize_factor*v1_n + v2_u_after
                    ball_2.velocity = amortize_factor*v2_n + v1_u_after
            
        for i in range(N):
            b = balls[i]
            for j in range(2):
                if b.position[j] < b.radius:
                    b.velocity[j] = 0
                    b.position[j] = b.radius          

                if b.position[j] > screen_limit[j] - b.radius:
                    b.velocity[j] = 0
                    b.position[j] = screen_limit[j] - b.radius
            
        count += 1
        for i in range(N):
            b = balls[i]
            if np.linalg.norm(b.velocity) < 0.05:
                b.velocity = np.array([0,0])
#             if count % 100 == 0:
#                 print(b)
    #       For debug
#         if count % 100 ==
        
        frames.append( copy.deepcopy(state) )
        converged = check_converge(frames)
        t += dt
        if t > 60:  # protection
            break;
    return state

In [307]:
def check_converge(frames):
    """
    Check if the final state has converged
    
    frames: List[State], the frames stocked in order of time
    """
    
    if len(frames) < 200:
        return False
    else:
        last_frames = frames[-200:]
        for frame in last_frames:
            for b in frame.balls:
                if np.linalg.norm(b.velocity) > 0.15:
                    return False
        return True

In [312]:
state_test_1 = State(200, 
                     400, 
                     [Ball(np.array([100, 215]), np.array([10, -10.0]), 20, color=(125, 0, 0)), 
                      Ball(np.array([100, 95]), np.array([0, 0]), 95, color=(0, 20, 126)),
                      Ball(np.array([20, 95]), np.array([10, 0]), 15, color=(129, 20, 16)),
                      Ball(np.array([150, 95]), np.array([-10, 0]), 35, color=(127, 0, 128)),], 
                     350)

In [313]:
evaluate_by_gravity(state_test_1)

In [310]:
print(state_test_1.balls[0])

Position, x: 180.0, y: 173.14368116847012 
Velocity, vx: 0.0, vy: -0.8429674054045778 
Radius: 20


In [314]:
state_test_1.plot_state()

Position, x: 162.13383831609383, y: 241.24818505018337 
Velocity, vx: 2.233550980688889, vy: -1.52602079405291 
Radius: 20
Position, x: 97.52216411468625, y: 146.1154595009114 
Velocity, vx: 0, vy: 0 
Radius: 95
Position, x: 15.0, y: 15.0 
Velocity, vx: 0, vy: 0 
Radius: 15
Position, x: 165.0, y: 35.0 
Velocity, vx: 0, vy: 0 
Radius: 35


In [6]:
class Game(object):
    
    """
    Implement the environment of the Game
    """
    
    def __init__(self, screen_x, screen_y, ball_setting ):
        """
        screen_x: float, the width of the screen
        screen_y: float, the height of the screen
        ball_setting: Dict( radius: float, reward: float ), the sizes of balls and corresponding rewards used in the function
        """
        
        self.screen_x = screen_x
        self.screen_y = screen_y
        self.ball_setting = ball_setting
        self.current_state = None  # TO DO
        self.init_state()
        
#         --TO ADD MORE--
        
    def init_state(self):
#         --TO DO--
        return 

    def check_fin(self):
#         --TO DO--
        return 
    
    def calculate_reward():
#         --TO DO--
        return 

#     ---TO ADD MORE---